In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

print "hello1"

/Users/skgill/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


hello1


In [3]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

# this is the y value: 
# rows ---> comment text, 
# columns --> "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
# [[0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [1 1 1 0 1 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]]

list_sentences_train = train["comment_text"]

# list_sentences_train
# 0    Explanation\nWhy the edits made under my usern...
# 1    D'aww! He matches this background colour I'm s...
# 2    Hey man, I'm really not trying to edit war. It...
# 3    "\nMore\nI can't make any real suggestions on ...
# 4    You, sir, are my hero. Any chance you remember...
# 5    "\n\nCongratulations from me as well, use the ...
# 6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
# 7    Your vandalism to the Matt Shirvington article...
# 8    Sorry if the word 'nonsense' was offensive to ...
# 9    alignment on this subject and which are contra...

list_sentences_test = test["comment_text"]

print "hello"

hello


In [4]:
max_features = 20000


# tokenizer: allows to vectorize a text corpus, 
# by turning each text into either a sequence of integers 
# (each integer being the index of a token in a dictionary) 

# num_words: the maximum number of words to keep, 
# based on word frequency. Only the most common num_words words will be kept
tokenizer = Tokenizer(num_words=max_features)

# assign each word an index
# {'sun': 1, 'september': 1, 'june': 1, 'other': 1, 'the': 1, 'and': 1, 
#  'like': 1, 'things': 1, 'in': 2, 'beautiful': 1, 'grey': 1, 'shining': 1,
#  'i': 1, 'is': 3, 'august': 1, 'it': 1, 'life': 1, 'this': 1}

# input : ["June is beautiful and I like it!"])

# output: [[5, 1, 11, 8, 14, 9, 16]]

tokenizer.fit_on_texts(list(list_sentences_train) + list(list_sentences_test))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)


# pad_sequences: Sequences that are shorter than maxlen are padded with value at the end.
# Sequences longer than maxlen are truncated so that they fit the desired length.
# By default, padded or truncated at the front

maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)


inp = Input(shape=(maxlen, ))
embed_size = 128

# Embedding: Turns positive integers (indexes) into dense vectors of fixed size.
# eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
# Embedding layer can only be used as the first layer in a model.

x = Embedding(max_features, embed_size)(inp)

# Sequence classification with LSTM
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)

# Dropout consists in randomly setting a fraction rate of input units to 0 
# at each update during training time, which helps prevent overfitting
x = Dropout(0.1)(x)

x = Dense(50, activation="relu")(x)


# relu: rectifier activation function
# 50: # of neurons in the first layers
# 0.1 (rate): float between 0 and 1. Fraction of the input units to drop.
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

# now the model will take as input arrays of shape (*, 200)
# and output arrays of shape (*, 32)

model = Model(inputs=inp, outputs=x)

# Configures the model for training.
# binary_crossentropy: logarithmic loss
# adam: stochastic gradient descent
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


# using mean square error to evalute the model instead
# use accuracy and mean absolute error as perfoemance metrics
# configure the model using the given metrics 
# model.compile(loss='mean_squared_error',
#               optimizer='sgd',
#               metrics=['mae', 'acc'])

print "done"

done


In [5]:
print("start fitting...")
model.fit(X_t,y, epochs=2, batch_size=32, validation_split=0.1)

print "done"

start fitting...
Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 2656s 18ms/step - loss: 0.0672 - acc: 0.9781 - val_loss: 0.0490 - val_acc: 0.9819
Epoch 2/2
143613/143613 [==============================] - 2404s 17ms/step - loss: 0.0444 - acc: 0.9832 - val_loss: 0.0479 - val_acc: 0.9821
done


In [6]:
print("start preditcting...")
y_pred = model.predict(X_te, batch_size=1024)
print "done"

start preditcting...
done


In [7]:
submission = pd.DataFrame.from_dict({'id': test['id']})
for idx, col in enumerate(list_classes):
    submission[col] = y_pred[:,idx]
submission.to_csv('submission.csv', index=False)
print "done"

done


In [1]:
r = model.to_json()
print r

In [5]:
from sklearn.metrics import log_loss
import sys, os, re, csv, codecs, numpy as np, pandas as pd

def calc_loss(y_true, y_pred):
    return np.mean([log_loss(y_true[:, i], y_pred[:, i]) 
                    for i in range(y_true.shape[1])])



In [17]:
true = pd.read_csv('input/train.csv')
pred = pd.read_csv('submission.csv')

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_true = true[list_classes].values
y_pred = pred[list_classes].values

print y_true[0]
print y_pred[0]

x = y_true - y_pred


print x[0]

print "done"

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
done
